In [84]:
#############################################################################
#This programm provides calculates BiasIndicator, Incredibility,Polarity,
#Subjectivity as percentofBiasIndicator for further graphing
#Article Incredibility, Polarity, Subjectivity taking in Article url as input 
#results are stored in Json and Excel
#to be modified to accept 'url' from 'input field'
#############################################################################

import pandas as pd
####import numpy as np
import requests
import matplotlib.pyplot as plt
plt.rcdefaults()

def getText(url):
    from newspaper import Article
    ### Getting the Text
    article = Article(url)
    article.download()
    article.parse()
    text = article.text
    return(text);
######################### Polarity and Subjectivity#################
def sentimentAnalysis(url):
    from textblob import TextBlob
    text = getText(url)
    ### analysing the text 
    testimonial = TextBlob(text)
    return(testimonial);  

#url = 'https://www.breitbart.com/2020-election/2020/10/03/as-trump-battles-covid-blm-antifa-disrupt-trump-rally-in-beverly-hills/'
url = 'https://www.bbc.co.uk/news/uk-54234084'
#url = 'https://metro.co.uk/2020/10/04/boris-johnson-admits-eat-out-to-help-out-could-have-helped-spread-covid-13370357/'
#url = 'https://www.foxnews.com/politics/trump-coronavirus-health-update-sunday-walter-reed'
#url = 'https://www.wsj.com/articles/covid-19-left-u-k-s-boris-johnson-and-brazils-jair-bolsonaro-unscathed-politically-11601739583'
#url = 'https://www.nbcnews.com/news/us-news/canadian-woman-charged-sending-ricin-laced-letter-white-house-n1240760'

getText(url)

polaritySubjectivity= pd.DataFrame(sentimentAnalysis(url).sentiment)  
polaritySubjectivity.rename(index={0:'Polarity',1:'Subjectivity'}, inplace=True)
polaritySubjectivityResults=polaritySubjectivity.T
print(polaritySubjectivityResults)
def getCredibilityScore(url):
####import requests
    Gate_Source_Cred_API = "https://cloud-api.gate.ac.uk/process-document/source-credibility?annotations=:DomainCredibility"
    reliability_txt = requests.post(Gate_Source_Cred_API, data = url, headers={'Content-Type': 'text/plain'}, 
                                    auth=('gc77cl10gevd', 'u7c5eeadladbtro9qul5'))
    ## Auth key - Jyothsna
    CredibilityText = reliability_txt.json()
    if (CredibilityText['entities'] != {}):
        for item in CredibilityText['entities']['DomainCredibility']:
            CredibilityScore = item.get('credibility-score', 0)
            CredibilityCategory = item.get('credibility-category', '')
            CredibilitySource = item.get('credibility-source', '')
            if item['credibility-source'] == 'Media Bias/Fact Check':
                return(url, CredibilityScore, CredibilityCategory, CredibilitySource)
    else:
      return("Not Available")    
    ## in case there is no 'Media Bias/Fact Check' but a different source:
    return(url, CredibilityScore, CredibilityCategory, CredibilitySource)

.55 * .75

   Polarity  Subjectivity
0      0.55          0.75


0.41250000000000003

In [82]:
######################### credibility #################
#getCredibilityScore(url)
credibility= pd.DataFrame(getCredibilityScore(url))  
credibility.rename(index={0:'Article',1:'CredibilityScore',2:'CredibilityCategory',3:'CredibilitySource'}, inplace=True)
credibilityResults=credibility.T

#############################################################

result = pd.concat([credibilityResults, polaritySubjectivityResults], axis=1, sort=False)

######################### BIAS Calculations #################
import matplotlib.pyplot as plt
plt.rcdefaults()

IncredibilityPercent = (100-(credibilityResults['CredibilityScore']))
PolarityPercent = ((polaritySubjectivityResults['Polarity'])*100)
SubjectivityPercent = ((polaritySubjectivityResults['Subjectivity'])*100)

TotalScore = (IncredibilityPercent+PolarityPercent+SubjectivityPercent)
BiasIndicator = ((IncredibilityPercent + abs(PolarityPercent) + SubjectivityPercent)/3)

ArticleIncredibilityInPercent = (100-(credibilityResults['CredibilityScore']))
ArticlePolarityBetweenMod1to0 = (polaritySubjectivityResults['Polarity'])
ArticleSubjectivityBetween0to1 = (polaritySubjectivityResults['Subjectivity'])

Incredibility = (IncredibilityPercent/ TotalScore)*100
Polarity = (PolarityPercent/ TotalScore)*100
Subjectivity = (SubjectivityPercent/ TotalScore)*100
result
polaritySubjectivityResults

,Polarity,Subjectivity
0,0.55,0.75


In [72]:
dfBias={'Article':url,'BiasIndicator':BiasIndicator,'Incredibility%' :Incredibility,
        'Polarity%' : Polarity, 'Subjectivity%':Subjectivity,
        ###'ArticleIncredibility': ArticleIncredibility, 
        ###'ArticlePolarity': ArticlePolarity, 
        ###'ArticleSubjectivity': ArticleSubjectivity
       }

### ANA
#result['BiasIndicator']  = BiasIndicator
#result['Incredibility%'] = Incredibility
#result['Polarity%']      = Polarity
#result['Subjectivity%']  = Subjectivity
#resultBREITBART = result

result= pd.DataFrame(dfBias)
#result.to_excel("BiasIndicatorValues.xlsx", sheet_name='results_1')  
#result.to_json(r'D:\AAtechreturners\sentimentalists\backend\BiasIndicatorValues.json')

###final_result = result
#final_result.append(result)
#final_result

result
#final_result = final_result.append(result, ignore_index=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

final_result

,Article,BiasIndicator,Incredibility%,Polarity%,Subjectivity%
0,https://www.breitbart.com/2020-election/2020/10/03/as-trump-battles-covid-blm-antifa-disrupt-trump-rally-in-beverly-hills/,31.6751,54.7524,-1.51352,46.7612
1,https://www.bbc.co.uk/news/uk-54234084,45,3.7037,40.7407,55.5556


In [69]:
result
final_result = result
final_result

,Article,BiasIndicator,Incredibility%,Polarity%,Subjectivity%
0,https://www.breitbart.com/2020-election/2020/10/03/as-trump-battles-covid-blm-antifa-disrupt-trump-rally-in-beverly-hills/,31.6751,54.7524,-1.51352,46.7612


In [ ]:
Breitbart
BBC
Metro
Fox News
Wall St Journal
NBC News


In [74]:
result

,Article,BiasIndicator,Incredibility%,Polarity%,Subjectivity%
0,https://www.bbc.co.uk/news/uk-54234084,45,3.7037,40.7407,55.5556


In [75]:
BiasIndicator

0    45
dtype: object

In [76]:
IncredibilityPercent 


0    5
Name: CredibilityScore, dtype: object

In [77]:
abs(PolarityPercent) 

0    55.0
Name: Polarity, dtype: float64

In [78]:
SubjectivityPercent

0    75.0
Name: Subjectivity, dtype: float64